In [1]:
import torch

## 1 Tensor

In [2]:
# 创建一个Tensor并设置requires_grad=True
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [3]:
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [4]:
print(x.is_leaf, y.is_leaf) # True False
# 像x这种直接创建的称为叶子节点，叶子节点对应的grad_fn是None
# y是通过一个加法操作创建的，所以它有一个为<AddBackward>的grad_fn

True False


In [5]:
z = y * y * 3
out = z.mean()
print(z,'\n', out) # out为为<MeanBackward>的grad_fn

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) 
 tensor(27., grad_fn=<MeanBackward0>)


In [6]:
# 通过.requires_grad_()改变requires_grad属性
a = torch.randn(2, 2) # 缺失情况下默认 requires_grad = False
a = ((a * 3) / (a - 1))
print(a.requires_grad) # False
a.requires_grad_(True)
print(a.requires_grad) # True
b = (a * a).sum()
print(b.grad_fn)

False
True


## 2 梯度

In [7]:
out.backward() # 等价于 out.backward(torch.tensor(1.))

In [8]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [9]:
# 再来反向传播一次，注意grad是累加的
out2 = x.sum() # 仅考虑标量对向量的求导，所以用sum或mean转换成标量
out2.backward() # 亦可写作out.sum().backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_() # 对于新关系，需要清除之前累计的梯度
out3.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])


In [10]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward0>)


In [11]:
# 现在 z 不是一个标量，所以在调用backward时需要传入
# 一个和z同形的权重向量进行加权求和得到一个标量
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


In [12]:
# 除了一些常用的如mean，sum, add等（应该都是）
# 还支持自定义函数（我的理解就是封装好的额）
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
print(d.grad_fn)

In [13]:
# 中断梯度
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2 
with torch.no_grad():
    y2 = x ** 3 # 此处可以直接采用y2 = (x ** 2).detach()
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [14]:
y3.backward() # 相当于y3 = x^2 + 常数
print(x.grad)

tensor(2.)


In [15]:
# 修改tensor的数值，但是又不希望被autograd记录（即不会影响反向传播），
# 那么我么可以对tensor.data进行操作
x = torch.ones(1, requires_grad=True)

print(x.data)  # 还是一个tensor
print(x.data.requires_grad)  # 但是已经是独立于计算图之外

y = 2 * x # 记录再计算图中
x.data *= 100  # 只改变了值，不会记录在计算图，所以不会影响梯度传播

y.backward()
print(x)  # 更改data的值也会影响tensor的值
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
